In [1]:
%load_ext autoreload
%autoreload 2

# qd_lasing_dynamics package Tutorial

This package is for analyzing quantum dot lasing dynamics in a Baysian framework.  It has objects for defining priors, the model, and doing Baysian parameter estimation with emcee.  

## Model Definition

Our theoretical model is the solution set to a set of three rate equations (see below). In this model, as is described in [1], it is assumed that carriers are injected into a quantum well and then are subsequently captured by quantum dots for radiative recombination. These coupled rate equations relate Photon Density, Quantum Dot occupation probability, and Carrier Density in the quantum well:

$$
\dot{S} = -\frac{S}{\tau_s} + g_0v(2\rho-1)S \tag{1}
$$

$$
\dot{\rho} = -\frac{\rho}{\tau_d} - g_0(2\rho-1)S + F(N,\rho) \tag{2}
$$

$$
\dot{N} = J - \frac{N}{\tau_n} - 2N_dF(N,\rho) \tag{3}
$$

Where S is photon density; N is carrier density in the quantum well; $\rho$ is the occupation probability in a quantum dot; $\tau_s$ is the photon lifetime; $\tau_n$ and $\tau_d$ are the carrier lifetimes in the well and the dot, respectively; $N_d$ is the 2D density of Quantum Dots, and J is the pump current density. $g_0 = \sigma_{res}v_g$, where $\sigma_{res}$ is the cross section of interaction of carriers in the dots with photons; $v_g$ is the group velocity; and $v = 2N_d\Gamma/d$, where $\Gamma$ is the confinement factor and $d$ is the thickness of the dot layer.

Additionally,

$$
F(N,\rho) = R_{cap}(1-\rho)-R_{esc}\rho \tag{4}
$$

Where $R_{cap} = CN^2 + BN$. C describes Auger carrier capture while B describes carrier-phonon capture. $R_{esc}$ is the rate of escape of carriers from quantum dots and is a temperature dependent function. 

By solving equations 1-3 in steady state, we can define S, $\rho$, and N in terms of input current, or J. Therefore, the S function in terms of J is essentially your "lasing curve". This curve will be what we are "fitting" our data to, as this is the format our data is in. 

When taking lasing curves on our devices, there is a gaussian distribution of noise with standard deviation X units, which will be our likelihood. Additionally, we will be assuming a flat prior for our C coefficient, or the Auger capture rate. Therefore, our statistical model will be:

$$
y_i = Normal(\mu, \sigma)
$$

$$
\mu = Steady state solution to equations 1-3
$$

$$
C \simeq Uniform(10^{-19}, 10^{-20})m^4/s
$$

## Objects

### Prior objects

Recall,

$$p(x) = \frac{1}{x_{\mathrm{max}}-x_{\mathrm{min}}}$$ (Uniform Prior)

$$p(x) = \frac{1}{x \,\ln\left(x_{\mathrm{max}}/x_{\mathrm{min}}\right)}$$ (Jeffery's Prior)

Here's how to use them:

In [2]:
from numpy import exp

from example.inference.model import JefferysPrior, UniformPrior, LogLikelihood

## Example of likelihood and posterior calculation using a Uniform Prior

In [6]:
import pandas as pd
dataHBL = pd.DataFrame({ 'x': [270526, 322632, 374737, 426842, 478947, 531053, 583158, 635263, 687368, 739474, 
                               791579, 843684, 895789, 947895, 1000000],
                         'y': [16320651054.799805 , 1.5083098256933988e+19, 4.088297611800159e+19, 6.922358218104778e+19, 
                               9.992340135914494e+19, 1.3280368221726885e+20, 1.677071608127957e+20, 2.0449461918170448e+20,
                               2.4304211200931892e+20, 2.832387185307339e+20, 3.249847029344283e+20, 3.6818999602051704e+20, 
                               4.127729320605625e+20, 4.586591902554361e+20, 5.057809018304661e+20],
                         'sigma_y': [2.18738E+19, 1.41619E+19, 1.94991E+19, 2.57733E+19, 3.71496E+19, 1.37598E+19, 
                                     2.5289E+19, 2.04495E+19, 2.18738E+19, 1.41619E+19, 1.94991E+19, 2.57733E+19, 
                                     3.71496E+19, 1.37598E+19, 2.5289E+19]})

x = dataHBL['x'][:]
y = dataHBL['y'][:]
sigma_y = dataHBL['sigma_y'][:]

print(LogLikelihood(1e-20, x, y, sigma_y).logllh())
print(LogLikelihood(1e-20, x, y, sigma_y).logpost(10e-21, 10e-19))

-681.4104499188221
-639.9538679090758


### References
1. O’Brien, D., Hegarty, S.P., Huyet, G. and Uskov, A.V., 2004. Sensitivity of quantum-dot semiconductor lasers to optical feedback. Optics letters, 29(10), pp.1072-1074.